该输出解析器允许用户以流行的 XML 格式从 LLM 获取结果。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 XML。




In [5]:
from langchain.output_parsers import XMLOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


In [16]:
actor_query = "生成汤姆·汉克斯的电影目录。"
output = model.invoke(
    f"""{actor_query}

响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
``` 
"""
)
print(output.content)

```xml
<xml>
  <movie>费城故事</movie>
  <movie>阿甘正传</movie>
  <movie>拯救大兵瑞恩</movie>
  <movie>绿里奇迹</movie>
  <movie>荒岛余生</movie>
  <movie>达芬奇密码</movie>
  <movie>幸福终点站</movie>
  <movie>云图</movie>
  <movie>菲利普船长</movie>
  <movie>萨利机长</movie>
  <movie>华盛顿邮报</movie>
  <movie>芬奇</movie>
</xml>
```


In [17]:
parser = XMLOutputParser()
parser.invoke(output)

{'xml': [{'movie': '费城故事'},
  {'movie': '阿甘正传'},
  {'movie': '拯救大兵瑞恩'},
  {'movie': '绿里奇迹'},
  {'movie': '荒岛余生'},
  {'movie': '达芬奇密码'},
  {'movie': '幸福终点站'},
  {'movie': '云图'},
  {'movie': '菲利普船长'},
  {'movie': '萨利机长'},
  {'movie': '华盛顿邮报'},
  {'movie': '芬奇'}]}

In [13]:
parser = XMLOutputParser()
# format_instructions =  parser.get_format_instructions()
format_instructions = """响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
```
"""
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions},
)

chain = prompt | model | parser

output = chain.invoke({"query": actor_query})
print(output)

{'xml': [{'movie': '费城故事'}, {'movie': '阿甘正传'}, {'movie': '拯救大兵瑞恩'}, {'movie': '绿里奇迹'}, {'movie': '西雅图未眠夜'}, {'movie': '达芬奇密码'}, {'movie': '幸福终点站'}, {'movie': '荒岛余生'}, {'movie': '猫鼠游戏'}, {'movie': '萨利机长'}]}


In [39]:
parser = XMLOutputParser()
format_instructions =  parser.get_format_instructions()
format_instructions

'The output should be formatted as a XML file.\n1. Output should conform to the tags below. \n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema. \n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'